In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import json
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import ast
from pathlib import Path
import pandas as pd

In [15]:
label_map = {
    'null': 0,'jogging': 1,'jogging (rotating arms)': 2,'jogging (skipping)': 3,'jogging (sidesteps)': 4,'jogging (butt-kicks)': 5,
    'stretching (triceps)': 6,'stretching (lunging)': 7,'stretching (shoulders)': 8,'stretching (hamstrings)': 9,'stretching (lumbar rotation)': 10,
    'push-ups': 11,'push-ups (complex)': 12,'sit-ups': 13,'sit-ups (complex)': 14,'burpees': 15,'lunges': 16,'lunges (complex)': 17,'bench-dips': 18
}

test_subjects = [21]
embedding_dim = 64
num_classes = len(label_map.keys())
sequence_length = 50
input_channels = 3
num_layers = 2
hidden_size = 128
learning_rate = 0.001
num_epochs = 50
batch_size = 32
test_subjects = [21]
loc = "right_arm"
THRESHOLD: float = 0.25
DEFAULT_CLASS: int = 0
device =  'cpu'# torch.device('cuda' if torch.cuda.is_available() else 'cpu')


RAW_DIR   = Path('data')
TRAIN_DIR = RAW_DIR / 'dataset_without_null'
TEST_CSV  = RAW_DIR / 'test.csv'                   # public test
LOCATION_IDS = ["right_arm", "left_arm", "right_leg", "left_leg"]
WORK_DIR = Path('work3')

In [16]:
class HARDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        x = np.array(ast.literal_eval(sample['x_axis']))
        y = np.array(ast.literal_eval(sample['y_axis']))
        z = np.array(ast.literal_eval(sample['z_axis']))

        data = np.vstack((x, y, z)).T
        label = int(sample['label'])

        return torch.FloatTensor(data), torch.LongTensor([label])

def load_and_split_data(data_dir: Path, loc: str):
    csv_path = data_dir / f"{loc}_windows.csv"
    df = pd.read_csv(csv_path)
    train_data = df[~df['sbj_id'].isin(test_subjects)]
    test_data = df[df['sbj_id'].isin(test_subjects)]

    train_dataset = HARDataset(train_data.to_dict('records'))
    test_dataset = HARDataset(test_data.to_dict('records'))

    return train_dataset, test_dataset

In [17]:
train_dataset, test_dataset = load_and_split_data(TRAIN_DIR,LOCATION_IDS[0])
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

train_loader

In [21]:

class Embedder(nn.Module):
    def __init__(self, input_channels=3, embed_dim=64):
        super(Embedder, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))

        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))

        self.conv3 = nn.Sequential(
            nn.Conv1d(64, embed_dim, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(embed_dim),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return x  # Output shape: (batch_size, embed_dim, reduced_sequence_length)

class DeepConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super(DeepConvLSTM, self).__init__()

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Combined model
class TimeSeriesClassifier(nn.Module):
    def __init__(self, embed_dim=64, lstm_hidden_dim=128, lstm_layers=2, num_classes=num_classes):
        super(TimeSeriesClassifier, self).__init__()
        self.embedder = Embedder(embed_dim=embed_dim)
        self.classifier = DeepConvLSTM(embed_dim, lstm_hidden_dim, lstm_layers, num_classes)

    def forward(self, x):
        features = self.embedder(x)
        logits = self.classifier(features)
        return logits


In [22]:
def predict_with_default(softmatrix: np.ndarray, threshold: float = 0.25, default_class: int = 0) -> np.ndarray:
    max_conf = softmatrix.max(axis=1)
    preds = softmatrix.argmax(axis=1)
    return np.where(max_conf < threshold, default_class, preds)

def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=0.001):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_acc = 0.0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs = inputs.permute(0, 2, 1).to(device)
            labels = labels.squeeze().to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.permute(0, 2, 1).to(device)
                labels = labels.squeeze().to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)

                # Apply threshold rule
                probs = torch.softmax(outputs, dim=1).cpu().numpy()
                predicted = predict_with_default(probs)

                total += labels.size(0)
                correct += (predicted == labels.cpu().numpy()).sum().item()

        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = correct / total

        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()

    model.load_state_dict(best_model_state)
    return model

def evaluate_model(model, test_loader, threshold=0.25):
    model.eval()
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.permute(0, 2, 1).to(device)
            labels = labels.squeeze().to(device)

            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            predicted = predict_with_default(probs, threshold=threshold)

            total += labels.size(0)
            correct += (predicted == labels.cpu().numpy()).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted)

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')

    return accuracy, all_labels, all_preds



In [24]:
model = TimeSeriesClassifier(
    embed_dim=embedding_dim,
    lstm_hidden_dim=hidden_size,
    lstm_layers=num_layers,
    num_classes=num_classes
)

model = train_model(
    model,
    train_loader,
    test_loader,
    num_epochs=num_epochs,
    learning_rate=learning_rate
)

Epoch 1/50: 100%|██████████| 2512/2512 [01:01<00:00, 40.71it/s]


Epoch 1: Train Loss: 1.3088, Val Loss: 1.0190, Val Acc: 0.6276


Epoch 2/50: 100%|██████████| 2512/2512 [01:01<00:00, 40.89it/s]


Epoch 2: Train Loss: 0.9705, Val Loss: 0.9114, Val Acc: 0.6601


Epoch 3/50: 100%|██████████| 2512/2512 [01:01<00:00, 40.55it/s]


...

Epoch 48: Train Loss: 0.1853, Val Loss: 1.1834, Val Acc: 0.7173


Epoch 49/50: 100%|██████████| 2512/2512 [01:02<00:00, 40.23it/s]


Epoch 49: Train Loss: 0.1829, Val Loss: 1.1394, Val Acc: 0.7296


Epoch 50/50: 100%|██████████| 2512/2512 [00:57<00:00, 44.05it/s]


Epoch 50: Train Loss: 0.1803, Val Loss: 1.2355, Val Acc: 0.7202


In [ ]:
test_acc, test_labels, test_preds = evaluate_model(model, test_loader)

model_save_path = WORK_DIR / f"model_{loc}.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

predictions_df = pd.DataFrame({
    'true_label': test_labels,
    'predicted_label': test_preds
})
predictions_save_path = WORK_DIR / f"predictions_{loc}.csv"
predictions_df.to_csv(predictions_save_path, index=False)
print(f"Predictions saved to {predictions_save_path}")